In [ ]:
import os
import sys
import torch
import random
import argparse
import numpy as np
import scipy
from matplotlib import pyplot
from jax.config import config
config.update("jax_enable_x64", True)

sys.path.append('../../timeseries_jax/')
sys.path.append('../../timeseries/experiments_scripts/')
sys.path.append('../../src_jax/')

from jax import numpy as jnp
import factor
import state_model
import observation_model
from ssm_em import StateSpaceEM

from scipy.stats import norm
from scipy.stats import zscore
from ssm_em import StateSpaceEM, load_model
from sklearn.covariance import EmpiricalCovariance
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.gaussian_process.kernels import ExpSineSquared, RBF

from exp_utils import *
import ssm
import statsmodels.api as sm

'''
sys.path.append('../../timeseries/kalman-jax-master')
from jax.experimental import optimizers
#from sde_gp import SDEGP
import approximate_inference as approx_inf
import priors
import likelihoods
from utils import softplus_list, plot
'''


class PredictiveDensity:
    def __init__(self, mu, sigma):
        if mu.ndim == 1:
            self.mu = np.array([mu]).T
        else:
            self.mu = np.array(mu)
        if sigma.ndim == 1:
            self.Sigma = np.array([sigma]).T
        else:    
            self.Sigma = np.array(sigma)
            
def reset_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
def train_linear_SSM(x_tr, dz):

    dx = x_tr.shape[1]
    sm = state_model.LinearStateModel(dz)
    om = observation_model.LinearObservationModel(dx, dz)
        
    ssm_em_lin = StateSpaceEM(jnp.array(x_tr), observation_model=om, state_model=sm)
    ssm_em_lin.run()
    
    return ssm_em_lin


def train_linear_hsk_SSM(x_tr, dz, du, init_lin_model):

    dx = x_tr.shape[1]
    sm_hs = state_model.LinearStateModel(dz)
    om_hs = observation_model.HCCovObservationModel(dx, dz, du)
    if init_lin_model == 1:
        ssm_lin = train_linear_SSM(x_tr, dz)
        om_hs.lin_om_init(ssm_lin.om)
    hs_model = StateSpaceEM(jnp.array(x_tr), observation_model=om_hs, state_model=sm_hs)
    hs_model.run()
    return hs_model

def train_nonlinear_SSM(x_tr, dz, **kwargs):
    '''
    to be updated; currently doesn't run
    LSEMStateModel -> sm_hs = state_model.LSEMStateModel(args.dz,args.dk# + param for basis func)
    '''
    nonlin_model = NonLinearStateSpace_EM(x_tr,args.dz, args.dk)
    nonlin_model.run()
    
    return nonlin_model

class HMM_class:
    
    def __init__(self, x_tr, K, obs_model='gaussian'):
        self.x_tr = x_tr
        self.D = x_tr.shape[1]
        self.K = K
        self.obs_model = obs_model
        self.model = self._train()
        
    def _train(self):
        model = ssm.HMM(self.K, self.D, observations=self.obs_model)
        model.fit(self.x_tr, method="em")
        return model

    def compute_predictive_log_likelihood(self, x_te):
        return self.model.log_likelihood(x_te)
    
    def compute_predictive_density(self, x_te):
        mask = np.logical_not(np.isnan(x_te))
        x_te_not_nan = np.zeros(x_te.shape)
        x_te_not_nan[mask] = x_te[mask]
        states = self.model.filter(x_te_not_nan, mask=mask)
        if self.obs_model == 'gaussian' or  self.obs_model == 'studentst':
            mean_te = np.dot(states, self.model.observations.mus)
        elif self.obs_model == 'ar':
            mean_te = np.sum(states[:,:,None] * (np.sum(self.model.observations.As[None] * x_te[:,None, None], axis=3) + self.model.observations.bs), axis=1)
        std_te = np.dot(states, np.sqrt(self.model.observations.Sigmas.diagonal(axis1=1, axis2=2)))
        print(mean_te.shape, std_te.shape)
        return PredictiveDensity(mean_te, std_te ** 2)

def train_HMM(x_tr, num_states, **kwargs):
    return HMM_class(x_tr, num_states)


class ARIMAX:
    
    def __init__(self, x_tr, p, q):
        self.x_tr = x_tr
        self.p = p
        self.q = q
        self._train()
        
    def _train(self):
        if x_tr.shape[1] == 1:
            self.mod = sm.tsa.statespace.SARIMAX(x_tr, trend='c', order=(self.p,0,self.q))
            self.fit_res = self.mod.fit(disp=False)
        else:
            self.mod = sm.tsa.VARMAX(x_tr, trend='c', order=(self.p,self.q))
            self.fit_res = self.mod.fit(disp=False, max_iter=1000)
            
    def compute_predictive_density(self, x_te):
        mod_te = self.mod.clone(x_te)
        res = mod_te.filter(self.fit_res.params)
        predict = res.get_prediction()
        predict_ci = predict.conf_int(alpha=1.-.68)
        mu = predict.predicted_mean
        if x_te.shape[1] == 1:
            std = predict.predicted_mean - predict_ci[:,0]
        else:
            std = predict.predicted_mean - predict_ci[:,:x_te.shape[1]]
        return PredictiveDensity(mu, std ** 2)
            
    def compute_predictive_log_likelihood(self, x_te):  
        mod_te = self.mod.clone(x_te)
        return mod_te.loglike(self.fit_res.params)
    
def train_arimax(x_tr, p, q,**kwargs):
    
    arimax_model = ARIMAX(x_tr, p, q)
    return arimax_model

In [ ]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_sunspots_e1(train_ratio=.75)
dz = 1
ssm_em_lin = train_linear_SSM(x_tr, dz)

In [ ]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_sunspots_e1(train_ratio=.75)
dz = 1
du = 1
ssm_em_hsk = train_linear_hsk_SSM(x_tr, dz, du, init_lin_model=True)

In [ ]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_sunspots_e1(train_ratio=.75)
K = 15
hmm = train_HMM(x_tr, K)

In [ ]:
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_sunspots_e1(train_ratio=.75)
p, q = 7, 5
arimax = train_arimax(x_tr, p, q)

In [ ]:
T_start = 5
T_stop = 695
seed = 0
reset_seeds(seed)
x_tr, x_va, x_te_original, x_te_na, s_tr_x = load_sunspots_e1(train_ratio=.75)
lin_pred_density = ssm_em_lin.compute_predictive_density(x_te_original)
lb_lin, ub_lin = lin_pred_density.mu[T_start:T_stop] - 1.68 * np.sqrt(lin_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2)), lin_pred_density.mu[T_start:T_stop] + 1.68 * np.sqrt(lin_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2))
lb_lin, ub_lin = s_tr_x.inverse_transform(lb_lin), s_tr_x.inverse_transform(ub_lin)
lb_lin = np.logaddexp(1, lb_lin)
ub_lin = np.logaddexp(1, ub_lin)
hsk_pred_density = ssm_em_hsk.compute_predictive_density(x_te_original)
lb_hsk, ub_hsk = hsk_pred_density.mu[T_start:T_stop] - 1.68 * np.sqrt(hsk_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2)), hsk_pred_density.mu[T_start:T_stop] + 1.68 * np.sqrt(hsk_pred_density.Sigma[T_start:T_stop].diagonal(axis1=1, axis2=2))
lb_hsk, ub_hsk = s_tr_x.inverse_transform(lb_hsk), s_tr_x.inverse_transform(ub_hsk)
lb_hsk = np.logaddexp(1, lb_hsk)
ub_hsk = np.logaddexp(1, ub_hsk)
hmm_pred_density = hmm.compute_predictive_density(x_te_original)
lb_hmm, ub_hmm = hmm_pred_density.mu[T_start:T_stop] - 1.68 * np.sqrt(hmm_pred_density.Sigma[T_start:T_stop]), hmm_pred_density.mu[T_start:T_stop] + 1.68 * np.sqrt(hmm_pred_density.Sigma[T_start:T_stop])
lb_hmm, ub_hmm = s_tr_x.inverse_transform(lb_hmm), s_tr_x.inverse_transform(ub_hmm)
lb_hmm = np.logaddexp(1, lb_hmm)
ub_hmm = np.logaddexp(1, ub_hmm)
arimax_pred_density = arimax.compute_predictive_density(x_te_original)
lb_arimax, ub_arimax = arimax_pred_density.mu[T_start:T_stop] - 1.68 * np.sqrt(arimax_pred_density.Sigma[T_start:T_stop]), arimax_pred_density.mu[T_start:T_stop] + 1.68 * np.sqrt(arimax_pred_density.Sigma[T_start:T_stop])
lb_arimax, ub_arimax = s_tr_x.inverse_transform(lb_arimax), s_tr_x.inverse_transform(ub_arimax)
lb_arimax = np.logaddexp(1, lb_arimax)
ub_arimax = np.logaddexp(1, ub_arimax)

In [ ]:
pyplot.rcdefaults()
pyplot.rc('font', size = 14)
fig = pyplot.figure(figsize=(16,3))
gs = fig.add_gridspec(1, 5, hspace=0)
x_te_plot = s_tr_x.inverse_transform(x_te_original)
x_te_plot = np.logaddexp(1, x_te_plot)

for dx in range(ssm_em_lin.Dx):
    ax1 = fig.add_subplot(gs[dx,:3])
    ax1.plot(range(T_start,T_stop), x_te_plot[T_start:T_stop, dx], 'k.', ms=1)
    ax1.fill_between(range(T_start,T_stop), lb_arimax[:,dx], ub_arimax[:,dx], color='C1', facecolor='none', alpha=1, lw=1)
    ax1.fill_between(range(T_start,T_stop), lb_lin[:,dx], ub_lin[:,dx], color='C0', alpha=.4, lw=0)
    ax1.fill_between(range(T_start,T_stop), lb_hmm[:,dx], ub_hmm[:,dx], color='C2', facecolor='none', alpha=.8, lw=1)
    ax1.fill_between(range(T_start,T_stop), lb_hsk[:,dx], ub_hsk[:,dx], color='C3', alpha=.4, lw=0)
    ax1.set_xlim([T_start,T_stop])
    ax1.set_xticks([])
    #ax1.set_yticks([])
    
    
    ax2 = fig.add_subplot(gs[dx,3])
    hsk_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hsk_pred_density.mu[:,dx]) / np.sqrt(hsk_pred_density.Sigma[:,dx,dx])))
    lin_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - lin_pred_density.mu[:,dx]) / np.sqrt(lin_pred_density.Sigma[:,dx,dx])))
    hmm_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hmm_pred_density.mu[:,dx]) / np.sqrt(hmm_pred_density.Sigma[:,dx])))
    arimax_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - arimax_pred_density.mu[:,dx]) / np.sqrt(arimax_pred_density.Sigma[:,dx])))
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), lin_cdf, color='C0', alpha=.7, lw=2)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), hmm_cdf, color='C2', alpha=.7, lw=2)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), hsk_cdf, color='C3', alpha=1)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), arimax_cdf, color='C1', alpha=.7)
    ax2.plot([0,1], [0,1], 'k--', lw=1., alpha=.5)
    ax2.set_xlim([0,1])
    ax2.set_ylim([0,1])
    ax2.set_xticks([])
    ax2.set_yticks([0,1])
    if dx == 0:
        ax2.set_yticklabels(labels=['','1'], fontsize=12)
    else:
        ax2.set_yticklabels(labels=['','0\n 1'], fontsize=12)
    
    ax3 = fig.add_subplot(gs[dx,4])
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (lin_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C0', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (hmm_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C2', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (arimax_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C1', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (hsk_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C3')
    ax3.plot([0,1], [0,0], 'k--', lw=1., alpha=.5)
    ax3.set_xlim([0,1])
    #ax2.set_ylim([0,1])
    ax3.set_xticks([])
    ax3.set_yticks([-10,10])
    ax3.set_ylim([-10,10])
    if dx == int(.5 * dx):
        ax2.set_ylabel('Model quantile')
        ax3.set_ylabel('$\Delta$ quantile [%]')
    if dx == 0:
        ax1.set_title('(a)', loc='left', fontweight='bold', x=-.1)
        ax2.set_title('(b)', loc='left', fontweight='bold', x=-.5)
        ax3.set_title('(c)', loc='left', fontweight='bold', x=-.5)

ax1.set_xticks(np.arange(0,len(x_te_original), 120))
ax1.set_xlim([10,len(x_te_original)-10])
ax1.set_xticklabels(np.arange(int(1983 - len(x_te_original)/12 + 1), int(1983), 10), rotation=45, fontsize=10)
ax1.set_xlabel('Years')
ax1.set_ylabel('Sunspot counts')
ax1.set_yticks([0,100,200,300])
ax2.set_xticks([0,1])
ax3.set_xticks([0,1])
ax2.set_xticks([0,1])
ax2.set_xlabel('Data quantile')
ax2.set_yticklabels(labels=['0','1'], fontsize=12)
ax3.set_xlabel('Data quantile')
pyplot.tight_layout()
fig.savefig('../../figs/aistats22/sunspots.pdf')
fig.savefig('../../figs/aistats22/sunspots.png', dpi=600)

In [ ]:
x_te_original.shape

In [ ]:
np.amax(s_tr_x.inverse_transform(x_tr))

In [ ]:
x_te_original = x_tr
lin_pred_density = ssm_em_lin.compute_predictive_density(x_te_original)
hsk_pred_density = ssm_em_hsk.compute_predictive_density(x_te_original)
hmm_pred_density = hmm.compute_predictive_density(x_te_original)
arimax_pred_density = arimax.compute_predictive_density(x_te_original)

In [ ]:
pyplot.rcdefaults()
pyplot.rc('font', size = 14)
fig = pyplot.figure(figsize=(16,4))
gs = fig.add_gridspec(1, 4, hspace=0)
T_start = 10
T_stop = 1680
for dx in range(ssm_em_lin.Dx):
    ax1 = fig.add_subplot(gs[dx,:2])
    ax1.plot(range(T_start,T_stop), x_te_original[T_start:T_stop, dx], 'k.', ms=1)
    lb_arimax = arimax_pred_density.mu[T_start:T_stop, dx] - 1.68 * np.sqrt(arimax_pred_density.Sigma[T_start:T_stop, dx])
    ub_arimax = arimax_pred_density.mu[T_start:T_stop, dx] + 1.68 * np.sqrt(arimax_pred_density.Sigma[T_start:T_stop, dx])
    ax1.fill_between(range(T_start,T_stop), lb_arimax, ub_arimax, color='C1', facecolor='none', alpha=1, lw=1)
    lb_lin = lin_pred_density.mu[T_start:T_stop, dx] - 1.68 * np.sqrt(lin_pred_density.Sigma[T_start:T_stop, dx, dx])
    ub_lin = lin_pred_density.mu[T_start:T_stop, dx] + 1.68 * np.sqrt(lin_pred_density.Sigma[T_start:T_stop, dx, dx])
    ax1.fill_between(range(T_start,T_stop), lb_lin, ub_lin, color='C0', alpha=.4, lw=0)
    lb_hmm = hmm_pred_density.mu[T_start:T_stop, dx] - 1.68 * np.sqrt(hmm_pred_density.Sigma[T_start:T_stop, dx])
    ub_hmm = hmm_pred_density.mu[T_start:T_stop, dx] + 1.68 * np.sqrt(hmm_pred_density.Sigma[T_start:T_stop, dx])
    ax1.fill_between(range(T_start,T_stop), lb_hmm, ub_hmm, color='C2', facecolor='none', alpha=.8, lw=1)
    lb_hsk = hsk_pred_density.mu[T_start:T_stop, dx] - 1.68 * np.sqrt(hsk_pred_density.Sigma[T_start:T_stop, dx, dx])
    ub_hsk = hsk_pred_density.mu[T_start:T_stop, dx] + 1.68 * np.sqrt(hsk_pred_density.Sigma[T_start:T_stop, dx, dx])
    ax1.fill_between(range(T_start,T_stop), lb_hsk, ub_hsk, color='C3', alpha=.4, lw=0)
    ax1.set_xlim([T_start,T_stop])
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_ylabel('$x_%d$' %(dx+1))
    
    ax2 = fig.add_subplot(gs[dx,2])
    hsk_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hsk_pred_density.mu[:,dx]) / np.sqrt(hsk_pred_density.Sigma[:,dx,dx])))
    lin_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - lin_pred_density.mu[:,dx]) / np.sqrt(lin_pred_density.Sigma[:,dx,dx])))
    hmm_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - hmm_pred_density.mu[:,dx]) / np.sqrt(hmm_pred_density.Sigma[:,dx])))
    arimax_cdf = np.sort(scipy.stats.norm.cdf((x_te_original[:,dx] - arimax_pred_density.mu[:,dx]) / np.sqrt(arimax_pred_density.Sigma[:,dx])))
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), lin_cdf, color='C0', alpha=.7, lw=2)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), hmm_cdf, color='C2', alpha=.7, lw=2)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), hsk_cdf, color='C3', alpha=1)
    ax2.plot(np.linspace(0,1,x_te_original.shape[0]), arimax_cdf, color='C1', alpha=.7)
    ax2.plot([0,1], [0,1], 'k--', lw=1., alpha=.5)
    ax2.set_xlim([0,1])
    ax2.set_ylim([0,1])
    ax2.set_xticks([])
    ax2.set_yticks([0,1])
    if dx == 0:
        ax2.set_yticklabels(labels=['','1'], fontsize=12)
    else:
        ax2.set_yticklabels(labels=['','0\n 1'], fontsize=12)
    
    ax3 = fig.add_subplot(gs[dx,3])
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (lin_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C0', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (hmm_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C2', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (arimax_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C1', alpha=.7, lw=2)
    ax3.plot(np.linspace(0,1,x_te_original.shape[0]), 100 * (hsk_cdf - np.linspace(0,1,x_te_original.shape[0])), color='C3')
    ax3.plot([0,1], [0,0], 'k--', lw=1., alpha=.5)
    ax3.set_xlim([0,1])
    #ax2.set_ylim([0,1])
    ax3.set_xticks([])
    ax3.set_yticks([-5,5])
    ax3.set_ylim([-7,7])
    if dx == 1:
        ax2.set_ylabel('Model quantile')
        ax3.set_ylabel('$\Delta$ quantile [%]')
    if dx == 0:
        ax1.set_title('(a)', loc='left', fontweight='bold', x=-.2)
        ax2.set_title('(b)', loc='left', fontweight='bold', x=-.5)
        ax3.set_title('(c)', loc='left', fontweight='bold', x=-.5)

ax1.set_xlabel('Timesteps')
ax2.set_xticks([0,1])
ax3.set_xticks([0,1])
ax2.set_xticks([0,1])
ax2.set_xlabel('Data quantile')
ax2.set_yticklabels(labels=['0','0\n 1'], fontsize=12)
ax3.set_xlabel('Data quantile')
pyplot.tight_layout()
fig.savefig('../../figs/aistats22/sunspots.pdf')
fig.savefig('../../figs/aistats22/sunspots.png', dpi=600)